# commands

> command functions.

In [ ]:
#| default_exp commands

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import yaml
import configparser
import typer
from rich.table import Table
from rich.console import Console
from rich.repr import auto as rich_auto
from dataclasses import dataclass, field

from typing import (
    ClassVar, List, Dict, Union, Optional, Tuple, Sequence, Set,
    Any, TypeVar, Type, Callable, cast, no_type_check
)

In [ ]:
#| export
from env2ini.constants import (
    PIP, DEP_SEP, PYPI_NAME, CONDA_NAME, 
    DEFAULT_DEPENDENCIES_TO_IGNORE, SETTINGS_INI,
    ENV_DEPENDENCIES, INI_REQUIREMENTS
)

from env2ini.utils import (
    split_package_str_at_first_non_alpha,
    to_macos_env_file, parse_aliases
)

from env2ini.types import (
    YamlDependencyStr, IniRequirementStr, YamlFileItem, 
    YamlDependencies, Dependencies, CondaDependencies, IniRequirements
)

from env2ini.dataclasses import (
    Dependency, CondaDependency, IniRequirement, DependencyList
)

from env2ini.rich import console, cprint, print_changes
from env2ini.typer import app

In [ ]:
#| export
@app.command()
def export_conda_to_ini(
    file: Optional[str] = typer.Option(
        'env.yml', '--file', '-f',
        help='YAML file to extract the new requirements from.',
    ),

    ini_file: Optional[str] = typer.Option(
        SETTINGS_INI, '--ini-file', '-i',
        help='The settings.ini file to update.',
    ),

    macos: Optional[bool] = typer.Option(
        False,'--macos', '-m',
        help='Whether or not to use the macos env file. `{os.path.basename(file)}.mac.yml`.',
    ),

    dryrun: Optional[bool] = typer.Option(
        False, '--dryrun', '-d',
        help='Whether to actually update the settings.ini file or just print the changes.',
    ),

    unchanged: Optional[bool] = typer.Option(
        False, '--unchanged', '-u', '-a',
        help='Whether to print all packages, including the ones whose versions haven\'t changed.',
    ),    

    dependencies_to_ignore: Optional[List[str]] = typer.Option(
        None, '--ignore', '-ig',
        help='List of dependencies to ignore.',
    ),

    dependency_aliases: Optional[str] = typer.Option(
        None, '--alias', '-al',
        help='Dictionary of aliases for conda dependencies in format: package1:alias1;package2:alias2.',
    ), 

    confirm: Optional[bool] = typer.Option(
        True, '--confirm', '-c',
        help='Ask for confirmation before writing to the ini file.',
    ),
    
):      
    # NOTE: notice that file is `env.mac.yml` and not `env.yml`. Now with Apple Silicon I have 
    #       one env file for more common CUDA versions and one for Apple Silicon.
    cprint('bold cyan', f'Loading environment yaml file {file}...')
    if macos:
        file = to_macos_env_file(file)
        
    with open(file, 'r') as f:
        yml_env = yaml.safe_load(f)
    
    yml_ignore = yml_env.get('env2ini', {}).get('ignore', [])
    ignore = DEFAULT_DEPENDENCIES_TO_IGNORE + (dependencies_to_ignore or []) + yml_ignore
    
    dependency_aliases = parse_aliases(dependency_aliases)
    yml_alias = yml_env.get('env2ini', {}).get('aliases', {})
    aliases = {**dependency_aliases, **yml_alias}

    # NOTE: read in the current dependencies from the conda env.yml file
    cprint('bold cyan', 'Extracting packages and their versions...')
    new_pkgs = CondaDependency.load_yml_dependencies(yml_env[ENV_DEPENDENCIES], ignore, aliases)

    # NOTE: read in the previous requirements from the settings.ini file
    cprint('bold cyan', 'Loading settings.ini file...')
    ini_cfg = configparser.ConfigParser()
    ini_cfg.read(ini_file)
    
    cprint('bold cyan', 'Comparing the old and new requirements...')    
    old_pkgs = IniRequirement.load_ini_requirements(ini_cfg['DEFAULT'][INI_REQUIREMENTS])

    # NOTE: check for changes
    added, removed, changed, remained = DependencyList.compare_requirements(old_pkgs, new_pkgs)

    # If --unchanged option is given, print unchanged packages as well
    if unchanged:
        print_changes(added, removed, changed, remained)

    else:
        print_changes(added, removed, changed, {})

    if dryrun:
        cprint('bold orange1', 'Dry Run: not updating the settings.ini file.')
        return
    
    if confirm:
        if not typer.confirm('Are you sure you want to write to the ini file?'):
            cprint('bold red', 'Cancelled writing to the ini file.')
            return
    
    # NOTE: update the requirements in the settings.ini file
    cprint('bold cyan', 'Updating the requirements...')    
    ini_cfg['DEFAULT'][INI_REQUIREMENTS] = CondaDependency.dump_ini_requirements(new_pkgs)

    cprint('bold cyan', 'Saving the updated settings.ini file...')
    with open(ini_file, 'w') as f:
        ini_cfg.write(f)

    cprint('bold green', 'Successfully updated the requirements in settings.ini!')

In [ ]:
#| export
def run_export_conda_to_ini():     
    app()    
    export_conda_to_ini()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()